In [1]:
import catboost
from catboost import CatBoostClassifier, Pool
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
#from sklearn.cross_validation import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from collections import Counter
from sklearn.svm import LinearSVC, SVC
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
df=pd.read_csv('train.csv')
y=df['ACTION']
X=df.drop(["ACTION"], axis=1)

In [3]:
cat_features = list(range(0, X.shape[1]))

In [4]:
print (cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
array=np.arange(0.05, .55, 0.05)
print(array)

[0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5 ]


In [7]:
import os
from tqdm import tqdm, trange

for i in array:
    cbc = CatBoostClassifier(
    boosting_type = 'Ordered',
    eval_metric= 'AUC',
    learning_rate= i,
     loss_function = 'Logloss',
     random_seed = 42,
     use_best_model= True

  
    )
    cbc.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    cat_features=cat_features,
    logging_level='Silent'
    )
    q=roc_auc_score(y_test, cbc.predict_proba(X_test)[:,1])
    if q>.898:
        print (f'learning rate {i} ROC_AUC {q}')
    i+=1
    

learning rate 0.1 ROC_AUC 0.8994544892902447


In [ ]:
cbc.score(X_test,y_test)

In [ ]:
prediction_hard = cbc.predict(X_test)
confusion_matrix(y_test, prediction_hard)

In [ ]:
print("ROC AUC score = ", roc_auc_score(y_test, cbc.predict_proba(X_test)[:,1]))

In [28]:
clf = cbc
pool = catboost.Pool(X_train, y_train, cat_features=cat_features)

feature_importance = clf.get_feature_importance(pool, fstr_type=catboost.EFstrType.FeatureImportance, prettified=True)
interactions = clf.get_feature_importance(pool, fstr_type=catboost.EFstrType.Interaction, prettified=True)
shap_values = clf.get_feature_importance(pool, fstr_type=catboost.EFstrType.ShapValues)

The model has complex ctrs, so the SHAP values will be calculated approximately.


In [30]:
feature_importance_df = pd.DataFrame(feature_importance, columns=['feature', 'importance'])

In [33]:
feature_importance_df.head(10)


,feature,importance
0,MGR_ID,31.979842
1,ROLE_FAMILY_DESC,16.131905
2,RESOURCE,13.078060
3,ROLE_DEPTNAME,11.380808
4,ROLE_ROLLUP_2,8.193116
5,ROLE_TITLE,6.736717
6,ROLE_CODE,5.851910
7,ROLE_FAMILY,4.146959
8,ROLE_ROLLUP_1,2.500683


In [36]:
total = feature_importance_df['importance'].sum()
print (total)

100.00000000000001


In [11]:
import pickle
with open('Catboost_opt', 'wb') as f:
    pickle.dump(model_full, f) 